<a href="https://colab.research.google.com/github/Location-Artistry/ColabNotebooks/blob/master/GOV_DATA_EXPLORER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EPA/EGLE Geo Data Explorer
## Working with ArcREST Endpoints from EPA ATTAINS and EGLE for Geo and Enviro Data   
### Read AGOL Data into GeoPandas, analyze and visualize with mapping libraries

## Create New MapServer Class
Supplemental library for exploring ArcGIS Online and Server hosted layers.  ArcGIS Python API has challenges accessing this data.  arcgis-restapi gives us the ability to get information about the MapServer Services and will use arcPy if possible, then use open source libraries:   
 'Python API for working with ArcGIS REST API, ArcGIS Online, and Portal/ArcGIS Enterprise.'    
 https://github.com/Bolton-and-Menk-GIS/restapi

In [24]:
class mapServer:
  # Class creates a MapServer object from the bmi-arcgis-restapi library
  # self.ags = instance of ArcServer object
  def __init__(self, url): 
    self.ags = restapi.ArcServer(url)
    self.url = url
    display('MAPSERVER OBJECT CREATED', self.ags)
  # getServices displays and returns list of all services
  def getServices(self):
    for i, service in enumerate(self.ags.services):
      display(f'{i} - Service Name: {service.name} -  Service Type: {service.type}')
    return self.ags.services 
  # getFolders displays and returns list of all folders
  def getFolders(self):
    for i, folder in enumerate(self.ags.folders):
      display(f'{i} - Folder: {folder}') 
    return self.ags.folders
  # services may be nested under folders, return all services in folders
  def getNestedServices(self):
    for i, folder in enumerate(self.ags.folders):
      display(f'{i} - Folder: {folder}')
      ags = restapi.ArcServer(f'{self.ags.url}/{folder}')
      for z, service in enumerate(ags.services):
        display(f'   {z} - Service Name: {service.name} -  Service Type: {service.type}')
  # list all atrributes for MapServer Object
  def getAttr(self):
    for attribute in dir(self.ags):
      display(f'Attribute Names: {attribute} - {self.ags[attribute]}')
  def listAttr(self):
    return dir(self.ags)
  # get subservice folder
  # working to create mapService from this function
  def getSubService(self, rootServer=0):
    # monitoring_thread = start_monitoring()
    folders = self.getFolders()
    services = self.getServices()
    if not folders:
      fID = int(input(f'Select index of subservice: '))
      serviceURL = services[fID].name
      subService = self.getService(serviceURL)
      return subService
    else:
      fID = int(input(f'Select index of subservice folder: '))
      serviceURL = (f'{self.url}/{folders[fID]}')
      subFolder = mapServer(serviceURL)
      subServices = subFolder.getServices()
      fID = int(input(f'Select index of subservice: '))
      serviceName = subServices[fID].name
      subService = mapService(self, serviceName)
      return subService

class mapService:
  # Class creates a <-MapService object-> from the bmi-arcgis-restapi library
  def __init__(self, server, servName): 
    self.ags = server.ags.getService(servName)
    #self.url = url
    display(f'MAPSERVICE OBJECT CREATED - {self.ags.url}')
  def getAttr(self):
    for attribute in dir(self.ags):
      display(f'Attribute Names: {attribute} - {self.ags[attribute]}')
  def listAttr(self):
    return dir(self.ags)
  def desc(self):
    return self.ags.description
  def spatialRef(self):
    return self.ags.spatialReference
  def units(self):
    return self.ags.units
  def layers(self):
    return self.ags.list_layers()
  def tables(self):
    return self.ags.list_tables()
  def getLayer(self):
    for i, lay in enumerate(self.ags.layers):
      display(f'{i} - {lay.name}')
    layers = self.layers()
    fID = int(input(f'Select index of Layer: '))
    layerObj = newLayer(self, layers[fID])
    return layerObj

class newLayer:
    # Class creates a <-Layer object-> from the bmi-arcgis-restapi library
  def __init__(self, service, lyrName): 
    self.lyr = service.ags.layer(lyrName)
    display(f'LAYER CREATED - {self.lyr.name} - id: {self.lyr.id} - url: {self.lyr.url}')
  def getInfo(self):
    display(f'{self.lyr.name} - subLayers: {self.lyr.subLayers} - capabilities: {self.lyr.capabilities} - query formats: {self.lyr.supportedQueryFormats} - count: {self.lyr.getCount()}')
  def getFields(self):
    return self.lyr.list_fields()
  def layerQuery (self):
    baseURL = self.lyr.url
    SQL = input(f'SQL query(ENTER for ALL): ') or '1=1'
    fields = input(f'Fields(ENTER for ALL)? ') or '*'
    RESTurl = (f'{baseURL}/query?f=geojson&where={SQL}&outFields={fields}')
    gdf = gpd.read_file(RESTurl)
    return gdf

## Entire Workflow for newly created Arc Server Workflow using bmi-arcgis-restapi


1.   Create MapServer with Server root url
2.   Derive specified MapService from root MapServer Object
3.   Generate layer from specific specified MapService Endpoint



In [ ]:
# 1 - Create MapServer with Server root url
rest_url = 'https://gispub.epa.gov/arcgis/rest/services'
SERVER = mapServer(rest_url)

In [26]:
# 2 - Derive specified MapService from root MapServer Object
subServer = SERVER.getSubService()

'0 - Folder: AgSTAR'

'1 - Folder: ER_Harvey'

'2 - Folder: monitor'

'3 - Folder: NELP'

'4 - Folder: NPDAT'

...

<ArcServer: "gispub.epa.gov" ("arcgis")>

'0 - Service Name: OW/AquaticNuisanceSpeciesLocator -  Service Type: MapServer'

'1 - Service Name: OW/ATTAINS_Assessment -  Service Type: MapServer'

Select index of subservice: 1


'MAPSERVICE OBJECT CREATED - https://gispub.epa.gov/arcgis/rest/services/OW/ATTAINS_Assessment/MapServer'

In [27]:
# 3 - Generate layer from specific specified MapService Endpoint
layerSer = subServer.getLayer()

'0 - ATTAINS Assessment Points'

'1 - ATTAINS Assessment Lines'

'2 - ATTAINS Assessment Areas'

'3 - ATTAINS Assessment Unit Catchment Associations'

Select index of Layer: 0


'LAYER CREATED - ATTAINS Assessment Points - id: 0 - url: https://gispub.epa.gov/arcgis/rest/services/OW/ATTAINS_Assessment/MapServer/0'

In [28]:
layerSer.getInfo()

'ATTAINS Assessment Points - subLayers: [] - capabilities: Map,Query,Data - query formats: JSON, geoJSON, PBF - count: 4022'

### Separate section using GeoPandas to read in the data directly from the ArcREST Endpoint

In [12]:
epaURL = 'https://gispub.epa.gov/arcgis/rest/services/OW/ATTAINS_Assessment/MapServer/0/query?f=geojson&where=1=1&outFields=*'
gdfEPA = gpd.read_file(epaURL)
gdfEPA.head()

,OBJECTID,submissionid,permid_joinkey,state,region,organizationid,orgtype,tas303d,organizationname,reportingcycle,assessmentunitidentifier,assessmentunitname,waterbodyreportlink,assmnt_joinkey,ircategory,overallstatus,isassessed,isimpaired,isthreatened,on303dlist,hastmdl,hasotherplan,visionpriority303d,drinkingwater_use,ecological_use,fishconsumption_use,recreation_use,other_use,algal_growth,ammonia,biotoxins,cause_unknown,cause_unknown_fish_kills,cause_unknown_impaired_biota,chlorine,dioxins,fish_consumption_advisory,flow_alterations,habitat_alterations,hydrologic_alteration,mercury,metals_other_than_mercury,noxious_aquatic_plants,nuisance_exotic_species,nuisance_native_species,nutrients,oil_and_grease,oxygen_depletion,other_cause,pathogens,pesticides,ph_acidity_caustic_conditions,polychlorinated_biphenyls_pcbs,radiation,solids_chlorides_sulfates,sediment,taste_color_and_odor,temperature,total_toxics,toxic_inorganics,toxic_organics,trash,turbidity,GLOBALID,geometry
0,1,{ea5304a8-04ea-089a-11f9-2c4dc0766643},{B5FB99AA-D105-802E-E053-FEDD43862C34},HI,09,21HI,State,Y,Hawaii,2020,HI138086,Hakalau Co. Pk.,https://mywaterway.epa.gov/waterbody-report/21...,{B5FB99A5-16BB-802E-E053-FEDD43862C34},2,Fully Supporting,Y,N,N,N,N,N,N,None,Not Assessed,None,Fully Supporting,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Meeting Criteria,None,None,None,None,None,None,None,None,None,None,None,None,None,{DCF54C6E-E635-429A-A235-65CA73174ABD},MULTIPOINT (-155.11961 19.88190)
1,2,{1b6f8338-ce5b-b915-b4eb-9a7c84157ddc},{B5FB99AA-DBB4-802E-E053-FEDD43862C34},SC,04,21SC60WQ,State,Y,South Carolina,2016,SCRS-06007,RS-06007,https://mywaterway.epa.gov/waterbody-report/21...,{B5FB99A7-82FD-802E-E053-FEDD43862C34},1,Fully Supporting,Y,N,N,N,N,N,N,None,None,None,Fully Supporting,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,{E05E1267-5661-405D-94EE-613D8239198D},MULTIPOINT (-81.67477 34.49325)
2,3,{ea5304a8-04ea-089a-11f9-2c4dc0766643},{B5FB99AA-DBB5-802E-E053-FEDD43862C34},HI,09,21HI,State,Y,Hawaii,2020,HI167153,Puamana Beach Co. Park,https://mywaterway.epa.gov/waterbody-report/21...,{B5FB99A7-834D-802E-E053-FEDD43862C34},2,Fully Supporting,Y,N,N,N,N,N,N,None,Not Assessed,None,Fully Supporting,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,{21FF59F9-AABC-49B8-A5D6-9A0455150934},MULTIPOINT (-156.67573 20.97868)
3,4,{1b6f8338-ce5b-b915-b4eb-9a7c84157ddc},{B5FB99AA-DBB7-802E-E053-FEDD43862C34},SC,04,21SC60WQ,State,Y,South Carolina,2016,SCMD-273,MD-273,https://mywaterway.epa.gov/waterbody-report/21...,{B5FB99A7-83E5-802E-E053-FEDD43862C34},1,Fully Supporting,Y,N,N,N,N,N,N,None,Fully Supporting,None,Fully Supporting,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,{9CA030FC-AF17-48A9-9D07-1D5F8230FDD7},MULTIPOINT (-80.12743 32.60801)
4,5,{1b6f8338-ce5b-b915-b4eb-9a7c84157ddc},{B5FB99AA-DBB8-802E-E053-FEDD43862C34},SC,04,21SC60WQ,State,Y,South Carolina,2016,SC05-02,05-02,https://mywaterway.epa.gov/waterbody-report/21...,{B5FB99A7-83E6-802E-E053-FEDD43862C34},1,Fully Supporting,Y,N,N,N,N,N,N,None,None,Fully Supporting,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,{2214E7E0-953F-4175-9116-311079BA1622},MULTIPOINT (-79.18339 33.30089)


## APPENDICIES

## Geopandas Libraries Install

In [ ]:
# Install and load libraries, may be able to remove some uneeded libraries
%%time 
!pip install bmi-arcgis-restapi

!apt install gdal-bin python-gdal python3-gdal 
!apt install python3-rtree 
!pip install git+git://github.com/geopandas/geopandas.git
!pip install descartes 
!pip install folium
!pip install pygeos
!pip install arcgis
!pip install contextily
!pip install xlsxwriter
import restapi
from pathlib import Path
import sys
import os
from zipfile import ZipFile
import datetime as dt
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import matplotlib
import matplotlib.pyplot as plt 
import folium
from IPython.display import display
import contextily as ctx
from IPython.display import Image
from IPython.display import Markdown as md